In [1]:
# !wget https://huggingface.co/datasets/mesolitica/mixtral-magicoder/resolve/main/post-postfilter.jsonl

In [2]:
from malaysian_sft import accept as accept_fn
import json
import random
from tqdm import tqdm
from glob import glob

In [3]:
system = [
    'always reply in english',
    'sentiasa respond dalam bahasa inggeris',
    'sentiasa jawab in english',
    'awak adalah chatbot yang respond dalam bahasa inggeris',
    'you are a chatbot that always respond in english',
    'u always reply in english',
    'awak sentiasa reply in english',
]

In [4]:
filtered = []

with open('post-postfilter.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        
        q = l['instruction_ms']
        if q is None:
            continue

        if l['answer'] is None:
            continue
            
        q = f'{random.choice(system)}\n\n{q.strip()}'
        
        d = {
            'prompt_input': None,
            'input': q,
            'output': l['answer'].strip(),
        }
        filtered.append(d)

132352it [00:05, 24905.25it/s]


In [5]:
len(filtered)

132352

In [6]:
files = glob('/home/husein/ssd3/camel-ai/*.translated.jsonl')

for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            
            try:
                
                if l['message_1_ms'] is None or l['message_2'] is None:
                    continue

                q = f"{random.choice(system)}\n\n{l['message_1_ms'].strip()}"

                d = {
                    'prompt_input': None,
                    'input': q,
                    'output': l['message_2'].strip(),
                }
                filtered.append(d)
            except:
                pass

50000it [00:01, 38490.41it/s]
78577it [00:00, 364368.66it/s]
20000it [00:00, 34818.24it/s]
20000it [00:00, 25669.70it/s]
20000it [00:00, 53800.89it/s]


In [7]:
len(filtered)

232928

In [8]:
skip = [
    'always better to find peaceful',
    'non-violent ways to express',
    'can lead to severe consequences',
    'still have questions or concerns about',
    'that doing so is illegal',
    'to report the incident to',
    'would recommend consulting with',
    'indonesian',
    'translates to',
    'idiom in',
    'in English',
    'in Malay',
    # 'language model'
]

break_at = [
    'help.openai.com',
    'openai',
    'cannot have personal opinions',
    's an ai language model',
    "i'm sorry",
    'many factors',
    'lgbt',
    'lesbian',
    'gender-neutral',
    'remain neutral',
    'without bias',
    'and neutral',
    'more inclusive',
    'neutrality',
    'non-bias',
    'discrimination',
    'avoid any forms of discrimination',
    'regardless of their gender',
    'inclusive and tolerant environment',
    'have personal views',
    'sexual orientation should be a top priority',
    's an objective ai',
    'avoid any forms of prejudice or hate',
    'regardless of their personal',
    'you understand this direction',
    'tolerant environment within ai',
    'cannot express my',
    'requires more context',
    'personal opinion',
    'have updated information',
    "don't have personal experiences",
    'there is no information',
    'tidak mempunyai akses kepada data atau maklumat',
    '10 april 2021',
    'ebagai model bahasa AI',
    'ebagai model bahasa ai',
    'model bahasa AI',
    'model bahasa ai',
    'bahasa ai',
    'ebagai model bahasa'
    'hat makes sense',
    'have access to data or information',
    'have access to the data or information',
    'hanya mempunyai akses kepada maklumat umum',
    'hanya boleh memberikan maklumat umum',
    'have personal preferences',
    'not have personal experiences',
    'not capable of having subjective opinions',
    'indonesian',
    'mistral',
    'terjemah',
    'translate'
]

In [9]:
roles = {
    'user': '<manusia>',
    'assistant': '<bot>'
}

files = glob('/home/husein/ssd2/soalan-augmentation/*conversation*.jsonl')
files.extend(glob('/home/husein/ssd2/soalan-augmentation/*multiturn*.jsonl'))
files = [f for f in files if 'rag' not in f]
for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            try:
                inputs = []
                for no, r in enumerate(l):
                    if no < (len(l) - 1):
                        if l[no + 1]['role'] == 'user':
                            next_text = l[no + 1].get('content_ms') or ''
                        else:
                            next_text = l[no + 1].get('content') or ''
                    else:
                        next_text = ''

                    if r['role'] == 'user':
                        q = r.get('content_ms', '') or ''
                        current_text = q       
                    else:
                        current_text = r.get('content') or ''

                    if l[no - 1]['role'] == 'user':
                        previous_text = l[no - 1].get('content_ms')
                    else:
                        previous_text = l[no - 1].get('content') or ''

                    # bad pairs
                    if r['role'] == 'user' and (len(current_text) < 2 or len(next_text) < 2):
                        # print('a', l, current_text, next_text, '\n')
                        continue
                    if r['role'] == 'assistant' and (len(current_text) < 2 or len(previous_text) < 2):
                        continue

                    # bad pairs
                    if r['role'] == 'user' and current_text[:20].lower() == next_text[:20].lower():
                        # print(no, r, current_text[:20].lower(), next_text[:20].lower())
                        continue
                    if r['role'] == 'assistant' and current_text[:20].lower() == previous_text[:20].lower():
                        # print(no, r, current_text[:20].lower(), previous_text[:20].lower())
                        continue

                    # remove alignments    
                    if r['role'] == 'user' and (any([b in current_text.lower() for b in break_at]) or any([b in next_text.lower() for b in break_at])):
                        # print(current_text, next_text)
                        break
                    if r['role'] == 'assistant' and (any([b in current_text.lower() for b in break_at]) or any([b in previous_text.lower() for b in break_at])):
                        # print(current_text, next_text)
                        break

                    role = roles[r['role']]

                    s = f"{role}: {current_text}"

                    inputs.append((s, r))

                if len(inputs) % 2 != 0:
                    inputs = inputs[:-1]

                if not len(inputs):
                    continue

                if len(inputs) < 2:
                    continue

                outputs = []
                for i in range(0, len(inputs), 2):

                    try:
                        content = inputs[i][1]['content']
                        content_ms = inputs[i][1]['content_ms']

                        if len(set(content_ms.split())) < (len(set(content.split())) / 2):
                            continue
                    except:
                        pass

                    try:
                        content = inputs[i + 1][1]['content']
                        content_ms = inputs[i + 1][1]['content_ms']

                        if len(set(content_ms.split())) < (len(set(content.split())) / 2):
                            continue
                    except:
                        pass

                    outputs.extend([
                        inputs[i][0],
                        inputs[i + 1][0]
                    ])

                if not len(outputs):
                    continue
                outputs[0] = outputs[0].replace('<manusia>: ', f'<manusia>: {random.choice(system)}\n\n')

                data = '\n'.join(outputs).strip()
                if not len(data):
                    continue

                d = {
                    'prompt_input': None,
                    'input': data,
                    'output': None,
                }
                filtered.append(d)
            except Exception as e:
                pass

60384it [00:30, 1951.61it/s]
48163it [00:20, 2372.18it/s]
57798it [00:03, 17237.42it/s]
103242it [02:17, 748.78it/s] 
135770it [04:32, 497.98it/s] 


In [10]:
len(filtered)

553223

In [12]:
filtered[-1]

{'prompt_input': None,
 'input': "<manusia>: you are a chatbot that always respond in english\n\nMengapa terdapat protes yang meluas dan rasa tidak puas hati orang ramai semasa Najib Razak memegang jawatan Perdana Menteri, dan bagaimana pentadbirannya bertindak balas terhadap protes ini?\n<bot>: During Najib Razak's tenure as Prime Minister of Malaysia from 2009 to 2018, there were several instances of widespread protests and public discontent. Here are some of the main reasons and the government's response:\n\n1. Bersih Rallies: The Bersih (Coalition for Clean and Fair Elections) movement organized several rallies demanding electoral reform, including free and fair elections, clean electoral roll, and the right to vote for all citizens. The first Bersih rally in 2007 saw a massive turnout of more than 40,000 people, despite a ban by the government. Subsequent Bersih rallies in 2011, 2012, and 2015 saw even larger crowds, with tens of thousands of people protesting against the governme

In [13]:
import pandas as pd

pd.DataFrame(filtered).to_parquet('prepared-force-english.parquet')

In [14]:
!ls -lh prepared-force-english.parquet

-rw-rw-r-- 1 husein husein 913M Dis  27 22:20 prepared-force-english.parquet


In [15]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="prepared-force-english.parquet",
    path_in_repo='data/force_english-00000-of-00001.parquet',
    repo_id="mesolitica/Malaysian-SFT",
    repo_type="dataset",
)

prepared-force-english.parquet:   0%|          | 0.00/956M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-SFT/commit/ece6aa6f631f175051ca2474489c4127f61d940d', commit_message='Upload data/force_english-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='ece6aa6f631f175051ca2474489c4127f61d940d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-SFT', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-SFT'), pr_revision=None, pr_num=None)